<a href="https://colab.research.google.com/github/lhiwi/complaint-rag-chatbot/blob/task1-eda/notebooks/eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## EDA

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Standard libraries
import os
import re

# Data handling
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Progress bars
from tqdm.auto import tqdm
tqdm.pandas()



In [ ]:
# load the data
RAW_PATH = '/content/drive/MyDrive/Copy of complaints.csv'

# Read file
df = pd.read_csv(RAW_PATH)

# Sanity-check
print(f"Loaded {df.shape[0]} rows × {df.shape[1]} cols from {RAW_PATH}")
df.head(3)



In [ ]:
for col in df.columns:
    print("  ", repr(col))

In [ ]:
# the five products of interest
TARGET_PRODUCTS = [
    'Credit card',
    'Personal loan',
    'Buy now pay later',
    'Savings account',
    'Money transfers'
]

# Text-cleaning function
def clean_text(txt):
    txt = txt.lower()
    txt = re.sub(r'http\S+', ' ', txt)
    txt = re.sub(r'writing to file a complaint', ' ', txt)
    txt = re.sub(r'[^a-z0-9\s]', ' ', txt)
    return re.sub(r'\s+', ' ', txt).strip()


In [ ]:
#  Load a sample with the actual column names
usecols = ['Date received', 'Product', 'Issue', 'Consumer complaint narrative']
df_sample = pd.read_csv(RAW_PATH, usecols=usecols, nrows=200_000)
df_sample = df_sample.dropna(subset=['Consumer complaint narrative'])
print(df_sample.shape)
print(f"Sample loaded: {df_sample.shape[0]} rows")
df_sample.head()

In [ ]:
# Drop empty narratives in the sample
df_sample = df_sample[df_sample['Consumer complaint narrative'].notna()
                      & df_sample['Consumer complaint narrative'].str.strip().ne('')]

df_sample.head()


In [ ]:
# Product distribution
prod_counts = df_sample['Product'].value_counts()
plt.figure(figsize=(10,5))
sns.barplot(x=prod_counts.index, y=prod_counts.values)
plt.xticks(rotation=45, ha='right')
plt.title('Sample: Complaints by Product')
plt.tight_layout()
plt.show()

In [ ]:
# Narrative lengths
df_sample['n_words'] = df_sample['Consumer complaint narrative'].str.split().apply(len)
plt.figure(figsize=(10,5))
sns.histplot(df_sample['n_words'], bins=50)
plt.xlim(0,500)
plt.title('Sample: Narrative Lengths (words)')
plt.tight_layout()
plt.show()

Time-Series of Complaint Volume

In [ ]:
# Identify any columns that look like a date
date_cols = [c for c in df_sample.columns if 'date' in c.lower()]
print("Found date columns:", date_cols)


In [ ]:
date_col = 'Date received'

if date_col in df_sample.columns:
    df_sample[date_col] = pd.to_datetime(df_sample[date_col], errors='coerce')
    monthly = (
        df_sample
          .dropna(subset=[date_col])
          .set_index(date_col)
          .resample('M')
          .size()
          .rename('complaint_count')
          .to_frame()
    )
    plt.figure(figsize=(10,4))
    sns.lineplot(data=monthly, x=monthly.index, y='complaint_count')
    plt.title('Sample: Monthly Complaint Volume')
    plt.xlabel('Month'); plt.ylabel('Count')
    plt.tight_layout(); plt.show()
else:
    print(f"No date column named {date_col!r}; skipping time-series plot.")


Top 10 Issue Labels

In [ ]:
top_issues = df_sample['Issue'].value_counts().nlargest(10)
plt.figure(figsize=(10,5))
sns.barplot(x=top_issues.values, y=top_issues.index, palette='mako')
plt.title('Sample: Top 10 Complaint Issue Labels')
plt.xlabel('Count'); plt.ylabel('Issue Label')
plt.tight_layout(); plt.show()


In [ ]:
# to save filtered data
OUT_PATH = 'data/filtered_complaints.csv'

Processing & Cleaning

In [ ]:
# Check the output directory exists
out_dir = os.path.dirname(OUT_PATH)  # this is 'data'
os.makedirs(out_dir, exist_ok=True)

# If you’d like to start fresh, delete any existing file
if os.path.exists(OUT_PATH):
    os.remove(OUT_PATH)

#  Read & process in chunks
reader = pd.read_csv(
    RAW_PATH,
    usecols=['Product', 'Consumer complaint narrative'],
    chunksize=500_000,
    iterator=True
)

for i, chunk in enumerate(tqdm(reader, desc='Processing chunks')):
    # 5.3.1 Filter to target products & non-empty narratives
    mask = (
        chunk['Product'].isin(TARGET_PRODUCTS)
        & chunk['Consumer complaint narrative'].notna()
        & chunk['Consumer complaint narrative'].str.strip().ne('')
    )
    df_chunk = chunk.loc[mask].copy()

    #  Clean text
    df_chunk['clean_narrative'] = df_chunk['Consumer complaint narrative']\
        .progress_apply(clean_text)

    #  Write out header only on first chunk
    df_chunk.to_csv(
        OUT_PATH,
        index=False,
        mode='w' if i == 0 else 'a',
        header=(i == 0)
    )

print(f" cleaned data saved to {OUT_PATH}")


In [ ]:
# Sanity-Check the Output
df_filt = pd.read_csv(OUT_PATH, nrows=5)
print(df_filt.shape)
df_filt.head()
